# Clean Historical Weather data and combine with Crime data

- Jim Haskin

- GA-Data Science
- Dec 2015

- 2/17/2016

## Method
- I collected historical weather data for the San Franciso area from January 2003 until December 2015. The data comes from Weather Underground. http://www.wunderground.com/history/
- I cleaned the data.
- To that data I added new features, such as the phase of the moon, that I wanted to investigate.
- I then combines the weather data with the crime data to form the final data file that is used for the analysis and modeling.



# Working Notes
## Clean up historical weather data
- wind_direction number - remove line breaks
- precipitation - change 'T' to .001 (trace)
- date - change to pandas time? use as index?

### catagoricals
- events

## features to add to the historical data
- temp/humidity
- wind chill effect
- sun_minutes
- moon_minutes
- moon phase
- daily deltas(swings) - temp, humidity, pressure
- day to day deltas of average/max/min - temp, humidity, pressure
- total_sun, total_moon - multiply cloudcover by minutes of sun/moon light
- 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
%matplotlib inline
import ephem as ep
from datetime import timedelta
from sf_sun_moon import sf_sun_moon

## Data source
- Data downloaded from WeatherUnderground by year and consolidated into a single file.
- sf_weather_2003_2015.csv
- http://www.wunderground.com/history/

FieldName|Type|Description                             
---------------|------------|---------------------
pst|string|Date in format : 2003-1-1
max_temperaturef|int|High Temperature for the day in degrees F
mean_temperaturef|int|Mean Temperature for the day in degrees F
min_temperaturef|int|Low Temperature for the day in degrees F
max_dew_pointf|int|High Dew Point for the day in degrees F 
meandew_pointf|int|Mean Dew Point for the day in degrees F
min_dewpointf|int|Low Dew Point for the day in degrees F
max_humidity|int|Maximum Humidity for the day in percentage 
mean_humidity|int|Mean Humidity for the day in percentage
min_humidity|int|Minimum Humidity for the day in percentage
max_sea_level_pressurein|float|High Sea Level for the day in inches
mean_sea_level_pressurein|float|Mean Sea Level for the day in inches
min_sea_level_pressurein|float|Low Sea Level for the day in inches
max_visibilitymiles|int|Maximum Visibility in miles
mean_visibilitymiles|int|Mean Visibility in miles
min_visibilitymiles|int|Minimum Visibility in miles
max_wind_speedmph|int|Maximum maintained Wind Speed in mph
mean_wind_speedmph|int|Mean maintained Wind Speed in mph
max_gust_speedmph|double|Maximum Wind Gust in mph
precipitationin|string|Amount is precipitation in inches
cloudcover|int|Cloud Cover, numeric ranges from 0 to 8
events|string|Significant Events such as 'Rain', 'Fog'
winddirdegrees<br_/>|string|General wind direction, numeric 0 359

In [2]:
!head -n 3 sf_weather_2003_2015.csv

pst,max_temperaturef,mean_temperaturef,min_temperaturef,max_dew_pointf,meandew_pointf,min_dewpointf,max_humidity,mean_humidity,min_humidity,max_sea_level_pressurein,mean_sea_level_pressurein,min_sea_level_pressurein,max_visibilitymiles,mean_visibilitymiles,min_visibilitymiles,max_wind_speedmph,mean_wind_speedmph,max_gust_speedmph,precipitationin,cloudcover,events,winddirdegrees<br_/>
2003-1-1,52,48,43,50,46,43,100,90,80,30.35,30.28,30.23,10,9,5,9,2,,0.0,3,,86<br />
2003-1-2,54,50,46,49,47,45,100,88,77,30.27,30.23,30.17,10,9,7,8,3,,0.0,5,,79<br />


### Read in data

In [3]:
w_data = pd.read_csv('sf_weather_2003_2015.csv')

In [4]:
w_data.head(3)

,pst,max_temperaturef,mean_temperaturef,min_temperaturef,max_dew_pointf,meandew_pointf,min_dewpointf,max_humidity,mean_humidity,min_humidity,...,max_visibilitymiles,mean_visibilitymiles,min_visibilitymiles,max_wind_speedmph,mean_wind_speedmph,max_gust_speedmph,precipitationin,cloudcover,events,winddirdegrees<br_/>
0,2003-1-1,52,48,43,50,46,43,100,90,80,...,10,9,5,9,2,NaN,0.0,3,NaN,86<br />
1,2003-1-2,54,50,46,49,47,45,100,88,77,...,10,9,7,8,3,NaN,0.0,5,NaN,79<br />
2,2003-1-3,55,50,46,48,46,44,97,84,67,...,10,10,7,8,3,NaN,0.0,4,NaN,56<br />


### Simplify feature labels

In [5]:
labels = ['pst', 'temp_max', 'temp_mean', 'temp_min', 'dew_point_max', 'dew_point_mean', 'dew_point_min',
          'humidity_max', 'humidity_mean', 'humidity_min', 'sl_pressure_max', 'sl_pressure_mean', 'sl_pressure_min',
          'visibility_max', 'visibility_mean', 'visibility_min', 'wind_speed_max', 'wind_speed_mean', 
          'wind_max_gust', 'precipitation', 'cloud_cover', 'events', 'wind_direction']

In [6]:
w_data.columns = labels
w_data.head(2)

,pst,temp_max,temp_mean,temp_min,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,humidity_min,...,visibility_max,visibility_mean,visibility_min,wind_speed_max,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,events,wind_direction
0,2003-1-1,52,48,43,50,46,43,100,90,80,...,10,9,5,9,2,NaN,0.0,3,NaN,86<br />
1,2003-1-2,54,50,46,49,47,45,100,88,77,...,10,9,7,8,3,NaN,0.0,5,NaN,79<br />


### Investigate data

In [7]:
w_data.describe()

,temp_max,temp_mean,temp_min,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,humidity_min,sl_pressure_max,sl_pressure_mean,sl_pressure_min,visibility_max,visibility_mean,visibility_min,wind_speed_max,wind_speed_mean,wind_max_gust,cloud_cover
count,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4283.000000,4748.000000
mean,65.848147,58.662174,51.172072,51.985889,48.254844,44.441870,88.037911,71.787911,53.988416,30.084261,30.027934,29.971131,9.973673,9.506108,7.837405,22.230413,9.837616,28.222041,3.614364
std,8.400893,6.530545,5.708427,5.621704,6.026269,7.453966,7.837339,10.197661,13.940999,0.140083,0.138543,0.142469,0.279829,1.124710,2.883617,7.087077,4.592513,8.963701,2.314393
min,45.000000,40.000000,32.000000,28.000000,16.000000,4.000000,40.000000,28.000000,10.000000,29.270000,29.030000,28.880000,4.000000,2.000000,0.000000,5.000000,0.000000,6.000000,0.000000
25%,60.000000,54.000000,47.000000,48.000000,45.000000,40.000000,83.000000,66.000000,46.000000,29.990000,29.940000,29.880000,10.000000,10.000000,7.000000,17.000000,6.000000,23.000000,2.000000
50%,65.000000,59.000000,52.000000,52.000000,49.000000,46.000000,89.000000,72.000000,55.000000,30.070000,30.010000,29.960000,10.000000,10.000000,9.000000,23.000000,10.000000,28.000000,4.000000
75%,71.000000,63.000000,55.000000,56.000000,53.000000,50.000000,93.000000,78.000000,63.000000,30.180000,30.120000,30.060000,10.000000,10.000000,10.000000,26.000000,13.000000,32.000000,6.000000
max,99.000000,82.000000,69.000000,67.000000,64.000000,62.000000,100.000000,100.000000,100.000000,30.590000,30.520000,30.480000,10.000000,10.000000,10.000000,60.000000,28.000000,204.000000,8.000000


In [8]:
w_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4748 entries, 0 to 4747
Data columns (total 23 columns):
pst                 4748 non-null object
temp_max            4748 non-null int64
temp_mean           4748 non-null int64
temp_min            4748 non-null int64
dew_point_max       4748 non-null int64
dew_point_mean      4748 non-null int64
dew_point_min       4748 non-null int64
humidity_max        4748 non-null int64
humidity_mean       4748 non-null int64
humidity_min        4748 non-null int64
sl_pressure_max     4748 non-null float64
sl_pressure_mean    4748 non-null float64
sl_pressure_min     4748 non-null float64
visibility_max      4748 non-null int64
visibility_mean     4748 non-null int64
visibility_min      4748 non-null int64
wind_speed_max      4748 non-null int64
wind_speed_mean     4748 non-null int64
wind_max_gust       4283 non-null float64
precipitation       4748 non-null object
cloud_cover         4748 non-null int64
events              1227 non-null object
wi

### Observations
- 4747 records
- Date in string form
- most measurements in int or float
- precipitation, wind_direction not numeric (fix)
- missing 465 wind_max_gust ~10%
- missing 3581 events ~75%. But will assume that a null is interpreted as no event happened

## Clean features

### Convert date to datetime

In [9]:
w_data['date'] = pd.to_datetime(w_data['pst'])

### Clean up wind_direction number
- line break was appended to number

In [10]:
w_data['wind_direction'] = w_data['wind_direction'].str.replace('<br />','').astype('int')
#w_data['wind_direction'] = w_data['wind_direction'].replace('<br_/>','',regex=True).astype('int')
#w_data['wind_direction'].astype(int, copy=True)

### Clean precipitation. 
- Has 'T' for trace instead of number
- 'T' forces the field to string type
- Replace trace with small numeric (0.005, half of lowest recorded)

In [11]:
w_data['precipitation'].value_counts().head(5)

0.00    3105
0.0      603
T        245
0.01      88
0.02      51
Name: precipitation, dtype: int64

In [12]:
w_data['precipitation'].dtype

dtype('O')

In [13]:
w_data['precipitation'] = w_data['precipitation'].str.replace('T','.005').astype('float')

### Cloud cover
- in weather underground cloud cover ranges from 0 to 8.( 4 mean 4/8) The forecast weather is in %
- convert to %

In [14]:
w_data['cloud_cover'] = (w_data['cloud_cover']*100/8).astype('int')

### Missing max_wind_gust
- ?????? not sure yet

## Catagoricals
### events
- records rain, fog, thunderstorm events
- will assume that nulls are days 'No' event occured
- since low occuring events are combinations of other events, consider manipulation to eliminate sparce events

In [15]:
w_data['events'].fillna('No', inplace=True)
w_data['events'] = w_data['events'].astype('category')

In [16]:
w_data['events'].value_counts()

No                   3521
Rain                  832
Fog                   323
Rain-Thunderstorm      42
Fog-Rain               26
Thunderstorm            4
dtype: int64

### Review data

In [17]:
w_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4748 entries, 0 to 4747
Data columns (total 24 columns):
pst                 4748 non-null object
temp_max            4748 non-null int64
temp_mean           4748 non-null int64
temp_min            4748 non-null int64
dew_point_max       4748 non-null int64
dew_point_mean      4748 non-null int64
dew_point_min       4748 non-null int64
humidity_max        4748 non-null int64
humidity_mean       4748 non-null int64
humidity_min        4748 non-null int64
sl_pressure_max     4748 non-null float64
sl_pressure_mean    4748 non-null float64
sl_pressure_min     4748 non-null float64
visibility_max      4748 non-null int64
visibility_mean     4748 non-null int64
visibility_min      4748 non-null int64
wind_speed_max      4748 non-null int64
wind_speed_mean     4748 non-null int64
wind_max_gust       4283 non-null float64
precipitation       4748 non-null float64
cloud_cover         4748 non-null int64
events              4748 non-null category

In [18]:
w_data.describe()

,temp_max,temp_mean,temp_min,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,humidity_min,sl_pressure_max,...,sl_pressure_min,visibility_max,visibility_mean,visibility_min,wind_speed_max,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,wind_direction
count,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,...,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4748.000000,4283.000000,4748.000000,4748.000000,4748.000000
mean,65.848147,58.662174,51.172072,51.985889,48.254844,44.441870,88.037911,71.787911,53.988416,30.084261,...,29.971131,9.973673,9.506108,7.837405,22.230413,9.837616,28.222041,0.047160,44.934288,246.112890
std,8.400893,6.530545,5.708427,5.621704,6.026269,7.453966,7.837339,10.197661,13.940999,0.140083,...,0.142469,0.279829,1.124710,2.883617,7.087077,4.592513,8.963701,0.191545,28.929665,71.714608
min,45.000000,40.000000,32.000000,28.000000,16.000000,4.000000,40.000000,28.000000,10.000000,29.270000,...,28.880000,4.000000,2.000000,0.000000,5.000000,0.000000,6.000000,0.000000,0.000000,1.000000
25%,60.000000,54.000000,47.000000,48.000000,45.000000,40.000000,83.000000,66.000000,46.000000,29.990000,...,29.880000,10.000000,10.000000,7.000000,17.000000,6.000000,23.000000,0.000000,25.000000,235.000000
50%,65.000000,59.000000,52.000000,52.000000,49.000000,46.000000,89.000000,72.000000,55.000000,30.070000,...,29.960000,10.000000,10.000000,9.000000,23.000000,10.000000,28.000000,0.000000,50.000000,273.000000
75%,71.000000,63.000000,55.000000,56.000000,53.000000,50.000000,93.000000,78.000000,63.000000,30.180000,...,30.060000,10.000000,10.000000,10.000000,26.000000,13.000000,32.000000,0.000000,75.000000,290.000000
max,99.000000,82.000000,69.000000,67.000000,64.000000,62.000000,100.000000,100.000000,100.000000,30.590000,...,30.480000,10.000000,10.000000,10.000000,60.000000,28.000000,204.000000,3.430000,100.000000,359.000000


## New features

### Sun and Moon information
- call subroutine to get:
- length of minutes of Sunlight for each day
- length of minutes of Moonlight for each day
- Phase of Moon

PROBLEM - values for moon length are over 24 hours check out

In [19]:
w_data['sun_length'], w_data['moon_phase'] = zip(*w_data['pst'].apply(sf_sun_moon))
#w_data['sun_length'], w_data['moon_length'], w_data['moon_phase'] = zip(*w_data['pst'].apply(sf_sun_moon))




### Daily Sun Level
- Use the cloud cover number and length of sun light to calculate a new feature, 'sun_units'

In [20]:
w_data['sun_units'] = w_data['sun_length'] * (1.0 - (w_data['cloud_cover']  * .01))

### Wind chill and Real feel
- Wind and humidity levels can effect how warm or cold you actually feel and effect peoples behavior .
- Create feature for wind chill (temp_mean/wind_speed_mean)
- Create feature for humidity/temp (temp_mean?????humidity_mean)

### Temperature and Humidy swings during the day
- Drastic changes in temp, pressure and humidity may effect peoples behavior.
- Create temp_delta
- Create humidity_delta 

In [21]:
w_data.insert(4, 'temp_delta', w_data['temp_max'] - w_data['temp_min'])

In [22]:
w_data.head()

,pst,temp_max,temp_mean,temp_min,temp_delta,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,...,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,events,wind_direction,date,sun_length,moon_phase,sun_units
0,2003-1-1,52,48,43,9,50,46,43,100,90,...,2,NaN,0.00,37,No,86,2003-01-01,578.733333,0.950084,364.602000
1,2003-1-2,54,50,46,8,49,47,45,100,88,...,3,NaN,0.00,62,No,79,2003-01-02,579.400000,0.983944,220.172000
2,2003-1-3,55,50,46,9,48,46,44,97,84,...,3,NaN,0.00,50,No,56,2003-01-03,580.100000,0.017763,290.050000
3,2003-1-4,57,52,48,9,54,50,46,100,91,...,3,NaN,0.00,50,Fog,295,2003-01-04,580.866667,0.051546,290.433333
4,2003-1-5,55,52,48,7,54,51,47,100,97,...,3,NaN,0.01,50,Fog,22,2003-01-05,581.700000,0.085328,290.850000


## Remove unwanted fields and write final data to csv file

In [23]:
w_data.set_index(w_data['date'], inplace=True)
w_data.head()

,pst,temp_max,temp_mean,temp_min,temp_delta,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,...,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,events,wind_direction,date,sun_length,moon_phase,sun_units
date,,,,,,,,,,,,,,,,,,,,,
2003-01-01,2003-1-1,52,48,43,9,50,46,43,100,90,...,2,NaN,0.00,37,No,86,2003-01-01,578.733333,0.950084,364.602000
2003-01-02,2003-1-2,54,50,46,8,49,47,45,100,88,...,3,NaN,0.00,62,No,79,2003-01-02,579.400000,0.983944,220.172000
2003-01-03,2003-1-3,55,50,46,9,48,46,44,97,84,...,3,NaN,0.00,50,No,56,2003-01-03,580.100000,0.017763,290.050000
2003-01-04,2003-1-4,57,52,48,9,54,50,46,100,91,...,3,NaN,0.00,50,Fog,295,2003-01-04,580.866667,0.051546,290.433333
2003-01-05,2003-1-5,55,52,48,7,54,51,47,100,97,...,3,NaN,0.01,50,Fog,22,2003-01-05,581.700000,0.085328,290.850000


In [24]:
data = w_data.drop(['pst', 'visibility_max', 'visibility_mean', 'visibility_min'], axis=1)
data.head()

,temp_max,temp_mean,temp_min,temp_delta,dew_point_max,dew_point_mean,dew_point_min,humidity_max,humidity_mean,humidity_min,...,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,events,wind_direction,date,sun_length,moon_phase,sun_units
date,,,,,,,,,,,,,,,,,,,,,
2003-01-01,52,48,43,9,50,46,43,100,90,80,...,2,NaN,0.00,37,No,86,2003-01-01,578.733333,0.950084,364.602000
2003-01-02,54,50,46,8,49,47,45,100,88,77,...,3,NaN,0.00,62,No,79,2003-01-02,579.400000,0.983944,220.172000
2003-01-03,55,50,46,9,48,46,44,97,84,67,...,3,NaN,0.00,50,No,56,2003-01-03,580.100000,0.017763,290.050000
2003-01-04,57,52,48,9,54,50,46,100,91,78,...,3,NaN,0.00,50,Fog,295,2003-01-04,580.866667,0.051546,290.433333
2003-01-05,55,52,48,7,54,51,47,100,97,80,...,3,NaN,0.01,50,Fog,22,2003-01-05,581.700000,0.085328,290.850000


In [25]:
data.to_csv('sf_weather_clean.csv')

## Combine Weather and Crime data and write to csv file

In [26]:
crime = pd.read_csv('sf_crime_clean.csv', index_col=0)
weather = data
#weather = pd.read_csv('sf_weather_clean.csv', index_col=0)

In [27]:
merge_data = crime.merge(weather, left_index=True, right_index=True)
data = merge_data.drop(['date'], axis=1)
merge_data.head()

,crime_level_sum,crime_level_count,crime_level,dayofweek,day,month,year,temp_max,temp_mean,temp_min,...,wind_speed_mean,wind_max_gust,precipitation,cloud_cover,events,wind_direction,date,sun_length,moon_phase,sun_units
2003-01-01,1254,622,10.000000,wednesday,1,1,2003,52,48,43,...,2,NaN,0.00,37,No,86,2003-01-01,578.733333,0.950084,364.602000
2003-01-02,750,411,5.974441,thursday,2,1,2003,54,50,46,...,3,NaN,0.00,62,No,79,2003-01-02,579.400000,0.983944,220.172000
2003-01-03,799,440,6.365815,friday,3,1,2003,55,50,46,...,3,NaN,0.00,50,No,56,2003-01-03,580.100000,0.017763,290.050000
2003-01-04,674,347,5.367412,saturday,4,1,2003,57,52,48,...,3,NaN,0.00,50,Fog,295,2003-01-04,580.866667,0.051546,290.433333
2003-01-05,755,377,6.014377,sunday,5,1,2003,55,52,48,...,3,NaN,0.01,50,Fog,22,2003-01-05,581.700000,0.085328,290.850000


In [28]:
data.to_csv('sf_crime_weather.csv')